# Wind farm

In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db
from pydae.grid_bpu import bpu

In [2]:
zeta = 0.1 # 1.0/np.sqrt(2) 
H_v = 4.0 
WB = 2 *np.pi* 50;
R_v = 0.0
X_v = 0.3

Lt = X_v 
P_max = 1/Lt
fn = np.sqrt(WB*P_max/(2*H_v))/(2*np.pi)

Dp = 0;
K_i = (2*np.pi*fn)**2/(WB*P_max);
K_g = Dp*K_i;
K_p = (2*zeta*2*np.pi*fn - K_g)/(WB*P_max);

T_q = 1.0/(2*np.pi*10/2)
K_q = (1.0 - 0.0)/(1.05 - 0.95)
K_i_q = 1e-6

theta_red = 3.0/180*np.pi
V = 1.0
p_ref = 0.9
q_ref = 0.434616
v_ref = 1.0
T_q = 1.0/(2*np.pi*10/2)

In [4]:
data = {
    "sys":{"name":"wind_farm","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.001},  
    "lines":[
        {"bus_j": "G01", "bus_k":"G01M", "X_pu":0.01, "R_pu":0.06, "S_mva": 2.5},
        {"bus_j": "G02", "bus_k":"G02M", "X_pu":0.01, "R_pu":0.06, "S_mva": 2.5},
        {"bus_j":"G01M", "bus_k":"POIM", "R_km":0.26, "X_km":0.12, "C_muF_km":0.25, "km":0.5},
        {"bus_j":"G02M", "bus_k":"POIM", "R_km":0.26, "X_km":0.12, "C_muF_km":0.25, "km":0.5},
        {"bus_j": "BES", "bus_k":"BESM", "X_pu":0.01, "R_pu":0.06, "S_mva": 5.0},
        {"bus_j":"BESM", "bus_k":"POIM", "R_km":0.26, "X_km":0.12, "C_muF_km":0.25, "km":0.5},
        {"bus_j":"POIM", "bus_k": "POI", "X_pu":0.01, "R_pu":0.05, "S_mva":10.0},
        {"bus_j": "POI", "bus_k":"GRID", "R_km":0.26, "X_km":0.12, "C_muF_km":0.25, "km":50.0, 'monitor':True}
         ],
    "buses":[
            {"name":   "G01", "P_W":0.0, "Q_var":0.0, "pos_x": -1500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":   "G02", "P_W":0.0, "Q_var":0.0, "pos_x": -1000.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":   "BES", "P_W":0.0, "Q_var":0.0, "pos_x":  -500.0, "pos_y":  200.0, "units": "m", "U_kV":0.69},
            {"name":  "G01M", "P_W":0.0, "Q_var":0.0, "pos_x": -1500.0, "pos_y":  180.0, "units": "m", "U_kV":20.0},
            {"name":  "G02M", "P_W":0.0, "Q_var":0.0, "pos_x": -1000.0, "pos_y":  180.0, "units": "m", "U_kV":20.0},
            {"name":  "BESM", "P_W":0.0, "Q_var":0.0, "pos_x":     0.0, "pos_y":    0.0, "units": "m", "U_kV":20.0},       
            {"name":  "POIM", "P_W":0.0, "Q_var":0.0, "pos_x":     0.0, "pos_y":    0.0, "units": "m", "U_kV":20.0},
            {"name":   "POI", "P_W":0.0, "Q_var":0.0, "pos_x":   100.0, "pos_y":    0.0, "units": "m", "U_kV":66.0},
            {"name":  "GRID", "P_W":0.0, "Q_var":0.0, "pos_x":   500.0, "pos_y":    0.0, "units": "m", "U_kV":66.0}
    ],
    "genapes":[{"bus":"POI","S_n":100e6,"F_n":50.0,"R_v":0.0,"X_v":0.001,"K_delta":0.001,"K_alpha":1.0}],
    "vsgs":[{"bus":"G01","type":"vsg_ll",'S_n':5e6,'F_n':50,'K_delta':0.0,
                 'R_v':R_v,'X_v':X_v,'K_p':K_p,'K_i':K_i,'K_g':K_g,'K_q':K_q,'T_q':T_q,'K_i_q':1e-6},
            {"bus":"G02","type":"vsg_ll",'S_n':5e6,'F_n':50,'K_delta':0.0,
                 'R_v':R_v,'X_v':X_v,'K_p':K_p,'K_i':K_i,'K_g':K_g,'K_q':K_q,'T_q':T_q,'K_i_q':1e-6},            
]}

bpu_obj = bpu(data_input=data)

g_list = bpu_obj.dae['g'] 
h_dict = bpu_obj.dae['h_dict']
f_list = bpu_obj.dae['f']
x_list = bpu_obj.dae['x']
params_dict = bpu_obj.dae['params_dict']


sys_dict = {'name':'wf2vsgll',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':bpu_obj.dae['y_ini'],
       'y_run_list':bpu_obj.dae['y_run'],
       'u_run_dict':bpu_obj.dae['u_run_dict'],
       'u_ini_dict':bpu_obj.dae['u_ini_dict'],
       'h_dict':h_dict}

bldr = db.builder(sys_dict);
bldr.build()